huggingface의 monologg/koelectra-base-v3-discriminator 모델을 사용하여 문제의 클래스를 예측하는 과정이 담긴 코드입니다.

#라이브러리 설치

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 1- 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np

data_path = '/content/drive/MyDrive/KMWP/code/data/oversampled_train.csv'

data = pd.read_csv(data_path)
data.head()

,problem,class
0,한 변의 길이가 24cm인 정육각형과 둘레가 같은 정팔각형이 있습니다. 이 정팔각형...,8
1,윤아는 부추전을 똑같이 8조각으로 나누어 한 조각을 먹었습니다. 윤미는 같은 크기의...,1
2,"화단 주변에 한 변이 12m인 정팔각형 모양의 울타리를 두른다면, 울타리는 모두 몇...",8
3,"6장의 숫자 카드 0, 9, 8, 7, 2, 1가 있습니다. 이를, 한 번씩 사용하...",3
4,0.26 x 0.8을 계산해 주세요.,1


In [ ]:
#data.drop(columns=['index', 'Unnamed: 5'], axis=1, inplace=True) 
#data.head()

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   problem  8000 non-null   object
 1   class    8000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 125.1+ KB


In [ ]:
data = data.astype({'class':'int32'})
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   problem  8000 non-null   object
 1   class    8000 non-null   int32 
dtypes: int32(1), object(1)
memory usage: 93.9+ KB


In [ ]:
# 데이터를 잘 섞어준다.
data = data.sample(frac=1).reset_index(drop=True)  # shuffling하고 index reset

In [ ]:
data.tail()

,problem,class
7995,어떤 수에 30를 더해야 할 것을 잘못하여 21을 곱했더니 105가 되었습니다. 바...,6
7996,"85, 3, 64, 36가 있습니다. 가장 큰 수를 가장 작은 수로 나눈 몫을 구하...",4
7997,"오늘 들어야 하는 수업은 수학, 음악, 영어, 과학, 사회 순서로 수업이 있습니다....",2
7998,"3 장의 숫자 카드 2, 3, 5 중에서 2장을 뽑아 두 수의 곱을 구하려고 합니다...",3
7999,어떤 양초에 불을 붙이고 50분이 지난 후에 길이를 재어 보니 123밀리미터 였습니...,5


주어진 test 데이터에는 class가 없기 때문에 정확도를 뽑기 위해 train 데이터를 분리하여 test_data를 따로 생성한다.

In [ ]:
data_length = int(len(data) * 0.9)

data_length

7200

In [ ]:
test_data = data[data_length:]
data = data[:data_length] 


len(data), len(test_data)

(7200, 800)

숫자형태의 카테고리로 인코딩을 수행한다.   
-> 그래야 loss가 잘 계산됨

In [ ]:
# label encoding 
from sklearn.preprocessing import LabelEncoder 

label_encoder = LabelEncoder() 
label_encoder.fit(data['class'])
num_labels = len(label_encoder.classes_) 

data['encoded_label'] = np.asarray(label_encoder.transform(data['class']), dtype=np.int32)
data.head() 

,problem,class,encoded_label
0,"어떤 수에 9를 더한 후 20를 곱하고, 50을 뺀 값을 2로 나누면 값이 125입...",6,5
1,1부터 9까지의 수 중에서 다음 식 33 > 5 * A 을 만족하는 가장 큰 수는 ...,3,2
2,3000 - 3100 - 3200 - A 를 규칙에 맞게 A에 알맞은 수를 구해보시오.,5,4
3,"어던 수를 10으로 나누었더니 7이 되었다면, 어떤 수는 얼마일까요?",6,5
4,"유진이의 키는 4/3m , 수진이의 키는 8/6m 입니다. 키가 더 큰 사람은 누구...",7,6


In [ ]:
problem = data.problem.to_list()
classes = data['encoded_label'].to_list() 

In [ ]:
problem[0], classes[:2]

('어떤 수에 9를 더한 후 20를 곱하고, 50을 뺀 값을 2로 나누면 값이 125입니다. 어떤 수를 구하시오.', [5, 2])

텍스트와 라벨을 따로 분리한다.

In [ ]:
from sklearn.model_selection import train_test_split 

train_prob, val_prob, train_class, val_class = train_test_split(problem,
                                                                classes, 
                                                                test_size=0.1, 
                                                                random_state=5)

In [ ]:
len(train_prob)

6480

# 2- 모델 및 토크나이저 불러오기
hugging face에서 monologg/koelectra-base-v3-discriminator 모델과   
Electra 토크나이저를 가져온다.

In [ ]:
from transformers import ElectraTokenizer

model_path = 'monologg/koelectra-base-v3-discriminator'

tokenizer = ElectraTokenizer.from_pretrained(model_path)

# 문제 토큰화
train_encodings = tokenizer(train_prob, truncation=True, padding=True)
val_encodings = tokenizer(val_prob, truncation=True, padding=True) 

토큰화 된 데이터셋을 Tensorflow의 Dataset object로 변환

In [ ]:
import tensorflow as tf  

# train set
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_class
))

# validation set
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings), 
    val_class
))

In [ ]:
train_dataset

<TensorSliceDataset element_spec=({'input_ids': TensorSpec(shape=(117,), dtype=tf.int32, name=None), 'token_type_ids': TensorSpec(shape=(117,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(117,), dtype=tf.int32, name=None)}, TensorSpec(shape=(), dtype=tf.int32, name=None))>

# 3- Koelectra fine-tuning 하기
Text Classification 이 목적이므로 TFElectraForSequenceClassification 클래스를 활용한다.

In [ ]:
from transformers import TFElectraForSequenceClassification

num_labels = len(label_encoder.classes_)

model = TFElectraForSequenceClassification.from_pretrained(model_path,
                                                           num_labels = num_labels,
                                                           from_pt=True)

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5) 
model.compile(optimizer = optimizer, )#loss=model.compute_loss, metrics=['accuracy'],)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFElectraForSequenceClassification: ['electra.embeddings.position_ids', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing TFElectraForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFElectraForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dens

In [ ]:
#!pip install pyyaml h5py

모델 학습시키기

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os 

callback_es = EarlyStopping(
    monitor = 'val_loss',
    patience = 2
)

mc = ModelCheckpoint('training_1/cp.ckpt', monitor='val_loss', mode='min', verbose = 1,
                     save_best_only=True)


model.fit(
    train_dataset.shuffle(2538).batch(16), epochs = 10, batch_size = 16,
    validation_data = val_dataset.shuffle(1000).batch(16),
    callbacks = [callback_es, mc]
)


Epoch 1/10
405/405 [==============================] - ETA: 0s - loss: 0.6129
Epoch 1: val_loss improved from inf to 0.24099, saving model to training_1/cp.ckpt


INFO:tensorflow:Assets written to: training_1/cp.ckpt/assets


INFO:tensorflow:Assets written to: training_1/cp.ckpt/assets


405/405 [==============================] - 162s 355ms/step - loss: 0.6129 - val_loss: 0.2410
Epoch 2/10
405/405 [==============================] - ETA: 0s - loss: 0.1741
Epoch 2: val_loss improved from 0.24099 to 0.12352, saving model to training_1/cp.ckpt


INFO:tensorflow:Assets written to: training_1/cp.ckpt/assets


...

INFO:tensorflow:Assets written to: training_1/cp.ckpt/assets


405/405 [==============================] - 138s 342ms/step - loss: 0.0564 - val_loss: 0.0669
Epoch 6/10
405/405 [==============================] - ETA: 0s - loss: 0.0582
Epoch 6: val_loss improved from 0.06690 to 0.05093, saving model to training_1/cp.ckpt


INFO:tensorflow:Assets written to: training_1/cp.ckpt/assets


INFO:tensorflow:Assets written to: training_1/cp.ckpt/assets


405/405 [==============================] - 139s 343ms/step - loss: 0.0582 - val_loss: 0.0509
Epoch 7/10
405/405 [==============================] - ETA: 0s - loss: 0.0356
Epoch 7: val_loss did not improve from 0.05093
405/405 [==============================] - 106s 261ms/step - loss: 0.0356 - val_loss: 0.0539
Epoch 8/10
405/405 [==============================] - ETA: 0s - loss: 0.0393
Epoch 8: val_loss did not improve from 0.05093
405/405 [==============================] - 106s 261ms/step - loss: 0.0393 - val_loss: 0.0601


In [ ]:
#dir(model)

In [ ]:
train_dataset.element_spec[0]['input_ids']

TensorSpec(shape=(117,), dtype=tf.int32, name=None)

원래는 아래의 코드에서 주석 처리된 부분을 실행해서, 인코딩 된 라벨들을 원상복귀시켜야 하는데,   
어찌된 일인지 계속 에러가 나서 실행하지 않았다.  

이 부분은 아래에서 다시 처리할 예정이다.

In [ ]:
import re

id2labels = model.config.id2label
#print(id2labels)
#model.config.id2label = {id: label_encoder.inverse_transform([int(re.sub('LABEL_', '', label))])[0] for id, label in id2labels.items()}
print(id2labels)

label2ids = model.config.label2id  
#model.config.label2id = {label_encoder.inverse_transform([int(re.sub('LABEL_', '', label))])[0] :id for id, label in id2labels.items()}
print(label2ids)

{0: 'LABEL_0', 1: 'LABEL_1', 2: 'LABEL_2', 3: 'LABEL_3', 4: 'LABEL_4', 5: 'LABEL_5', 6: 'LABEL_6', 7: 'LABEL_7'}
{'LABEL_0': 0, 'LABEL_1': 1, 'LABEL_2': 2, 'LABEL_3': 3, 'LABEL_4': 4, 'LABEL_5': 5, 'LABEL_6': 6, 'LABEL_7': 7}


모델 저장하기

In [ ]:
model.save_pretrained('finetuned_koelectra.h5')
tokenizer.save_pretrained('finetuned_koelectra')

('finetuned_koelectra/tokenizer_config.json',
 'finetuned_koelectra/special_tokens_map.json',
 'finetuned_koelectra/vocab.txt',
 'finetuned_koelectra/added_tokens.json')

# 4- 저장된 모델 불러와서 클래스 예측하기

In [ ]:
# 위에서 분리한 test_data를 가져온다
test_data.head() 

,problem,class
7200,"학교에서 국어, 수학, 영어, 과학, 사회의 순서로 시험을 봤습니다. 세번째로 시험...",2
7201,"가로가 세로의 1/3배인 직사각형의 가로가 1.23m 일 때, 넓이는 몇 제곱미터인...",8
7202,"경진이는 1/5시간 동안, 인수는 1/4시간 동안 운동을 했습니다. 누가 운동울 더...",7
7203,12 이상 24 이하인 자연수는 모두 몇 개일 지 구해보시오.,2
7204,삼각형의 한 각의 크기가 75°일 때 나머지 두각의 합을 구하시오.,8


In [ ]:
# 위에서 저장한 모델 및 토크나이저 가져오기
from transformers import TextClassificationPipeline 

loaded_tokenizer = ElectraTokenizer.from_pretrained('finetuned_koelectra')
loaded_model = TFElectraForSequenceClassification.from_pretrained('finetuned_koelectra.h5')

text_classifier = TextClassificationPipeline(
    tokenizer = loaded_tokenizer,
    model=loaded_model,
    framework='tf',
    return_all_scores=True
)


All model checkpoint layers were used when initializing TFElectraForSequenceClassification.

All the layers of TFElectraForSequenceClassification were initialized from the model checkpoint at finetuned_koelectra.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFElectraForSequenceClassification for predictions without further training.


from_pretrained() 메소드에 저장된 모델 디렉토리를 넣어 fine-tuning한 model 및 tokenizer를 가져올 수 있다.  

transformers의 Pipelines 클래스를 사용하면 특정 task에 대한 inference를 수행할 수 있다.

In [ ]:
# 클래스 예측하기
predicted_label_list = [] 
predicted_score_list = [] 

for text in test_data['problem']: 
  #predict 
  preds_list = text_classifier(text)[0]

  sorted_preds_list = sorted(preds_list, key=lambda x: x['score'], reverse=True)
  
  predicted_label_list.append(sorted_preds_list[0]['label']) 
  predicted_score_list.append(sorted_preds_list[0]['score'])

결과가 어떻게 나오는지 확인

In [ ]:
preds_list

[{'label': 'LABEL_0', 'score': 0.9948701858520508},
 {'label': 'LABEL_1', 'score': 0.00015562408952973783},
 {'label': 'LABEL_2', 'score': 2.824341572704725e-05},
 {'label': 'LABEL_3', 'score': 9.84358339337632e-05},
 {'label': 'LABEL_4', 'score': 0.0018363650888204575},
 {'label': 'LABEL_5', 'score': 0.0003783324209507555},
 {'label': 'LABEL_6', 'score': 3.665665281005204e-05},
 {'label': 'LABEL_7', 'score': 0.002596199745312333}]

In [ ]:
preds_list[0].items()

dict_items([('label', 'LABEL_0'), ('score', 0.9948701858520508)])

In [ ]:
len(predicted_label_list)
predicted_label_list[:5]

['LABEL_1', 'LABEL_7', 'LABEL_6', 'LABEL_1', 'LABEL_7']

In [ ]:
predicted_score_list[:5]

[0.999885082244873,
 0.9987512826919556,
 0.9995552897453308,
 0.9996449947357178,
 0.9983915686607361]

위에서 인코딩한 라벨을 처리해준다.   
LABEL_을 빼고 1을 더해 원래 class 값으로 저장한다.

In [ ]:
predicted_label_list_2 = []
for i in range(len(predicted_label_list)):
  predicted_label_list_2.append(re.sub('LABEL_', '', predicted_label_list[i]))
  predicted_label_list_2= list(map(int, predicted_label_list_2))
  predicted_label_list_2[i] += 1
predicted_label_list_2[:5]

[2, 8, 7, 2, 8]

In [ ]:
len(sorted_preds_list)

8

test_data에 예측한 클래스와 확률값을 추가하여 확인한다.

In [ ]:
test_data['pred'] = predicted_label_list_2 
test_data['score'] = predicted_score_list
test_data.head()

,problem,class,pred,score
7200,"학교에서 국어, 수학, 영어, 과학, 사회의 순서로 시험을 봤습니다. 세번째로 시험...",2,2,0.999885
7201,"가로가 세로의 1/3배인 직사각형의 가로가 1.23m 일 때, 넓이는 몇 제곱미터인...",8,8,0.998751
7202,"경진이는 1/5시간 동안, 인수는 1/4시간 동안 운동을 했습니다. 누가 운동울 더...",7,7,0.999555
7203,12 이상 24 이하인 자연수는 모두 몇 개일 지 구해보시오.,2,2,0.999645
7204,삼각형의 한 각의 크기가 75°일 때 나머지 두각의 합을 구하시오.,8,8,0.998392


# 5- 평가하기

In [ ]:
from sklearn.metrics import classification_report 

print(classification_report(y_true=test_data['class'], y_pred=test_data['pred']))

              precision    recall  f1-score   support

           1       0.97      0.94      0.96        90
           2       1.00      1.00      1.00       107
           3       0.98      0.99      0.98        96
           4       0.99      0.98      0.98        94
           5       0.98      0.99      0.98        96
           6       0.98      0.99      0.98        98
           7       0.99      1.00      1.00       104
           8       0.98      0.97      0.98       115

    accuracy                           0.98       800
   macro avg       0.98      0.98      0.98       800
weighted avg       0.98      0.98      0.98       800



- 데이터 불균형을 해결한 데이터로 평가한 결과 정확도 98%
